In [1]:
import pandas as pd
import requests
import json
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine

load_dotenv()
DATABASE_CONNECTION_URI = os.environ["DB_URL"]
API_KEY = os.environ["API_KEY"]
DOMAIN = os.environ["DOMAIN"]
PASSWORD = os.environ["PASSWORD"]

# create a connection to the database
engine = create_engine(DATABASE_CONNECTION_URI)

# DB Schema Handling
api_schema = json.load(open("api_schema.json"))["API"]
freshdesk_schema = api_schema["Freshdesk"]

class FreshdeskAPI:
    def __init__(self):
        # IDs
        self.api_key = API_KEY
        self.domain = DOMAIN
        self.password = PASSWORD

    def raw_export_tickets(self, properties):
        q_results = 101
        page = 1
        results = []
        while q_results >= 100:
            api_response = requests.get(
                "https://" + self.domain + ".freshdesk.com/api/v2/tickets?per_page=100&updated_since=2023-01-01&include=stats,requester&page=" + str(
                    page),
                auth=(self.api_key, self.password))
            api_response = json.loads(api_response.content)
            q_results = len(api_response)
            page += 1
            for index, d in enumerate(api_response):
                for prop in list(d):
                    if type(d[prop]) is dict:
                        for item in list(d[prop]):
                            api_response[index].update({prop + "_" + item: api_response[index][prop][item]})
                        api_response[index].pop(prop)
            results.extend(api_response)
            print("Freshdesk Tickets Export has gathered " + str(len(results)) + " Tickets")
        target_results = []
        for index, d in enumerate(results):
            d = {key: value for key, value in results[index].items() if key in properties}
            target_results.append(d)
        df = pd.DataFrame(target_results)
        # Constant Values Replacement
        source_values = {"1": 'Email',
                         "2": 'Portal',
                         "3": 'Phone',
                         "7": 'Chat',
                         "9": 'Feedback Widget',
                         "10": 'Outbound Email'}
        status_values = {"2": "Open",
                         "14": "Infraestructura",
                         "23": "Infraestructura - Actualización",
                         "3": "Pending",
                         "4": "Resolved",
                         "5": "Closed",
                         "6": "Esperando al Cliente",
                         "10": "Capacitación agendada",
                         "15": "Esperando subscripción del cliente al servicio",
                         "16": "Esperando aprobación de presupuesto por repuesto",
                         "17": "SLA Itau",
                         "18": "Esperando Hardware para Taller",
                         "19": "Envio de Repuesto",
                         "21": "Pendiente de Actualizacion",
                         "22": "En Desarrollo",
                         "24": "Soporte Mediterránea"}
        group_values = {"9000176648.0": "Aplicativos de Usuarios",
                        "9000161122.0": "Cámara IP",
                        "9000160962.0": "Debmedia Nivel 1",
                        "9000168495.0": "Debmedia Nivel 2",
                        "9000168511.0": "Debmedia Nivel 3",
                        "9000176646.0": "Gestión de Calidad",
                        "9000174341.0": "Soporte-Desarrollo Nivel 3",
                        "9000174342.0": "Soporte-Infraestructura Nivel 3",
                        "9000176647.0": "Talento Humano",
                        "9000162659.0": "Ticket Anulado",
                        "9000176648": "Aplicativos de Usuarios",
                        "9000161122": "Cámara IP",
                        "9000160962": "Debmedia Nivel 1",
                        "9000168495": "Debmedia Nivel 2",
                        "9000168511": "Debmedia Nivel 3",
                        "9000176646": "Gestión de Calidad",
                        "9000174341": "Soporte-Desarrollo Nivel 3",
                        "9000174342": "Soporte-Infraestructura Nivel 3",
                        "9000176647": "Talento Humano",
                        "9000162659": "Ticket Anulado"
                        }
        df["source"] = df["source"].astype(str)
        df["source"].replace(source_values, inplace=True)
        df["status"] = df["status"].astype(str)
        df["status"].replace(status_values, inplace=True)
        df["group_id"] = df["group_id"].astype(str)
        df["group_id"].replace(group_values, inplace=True)
        # print(df.dtypes)
        df.to_csv("raw_freshdesk_tickets.csv", encoding="latin-1", index=False, errors='ignore')

    def raw_export_surveys(self, properties):
        q_results = 101
        page = 1
        results = []
        while q_results >= 100:
            api_response = requests.get(
                "https://" + self.domain + ".freshdesk.com/api/v2/surveys/satisfaction_ratings?per_page=100&created_since=2023-01-01&page=" + str(
                    page),
                auth=(self.api_key, self.password))
            api_response = json.loads(api_response.content)
            q_results = len(api_response)
            page += 1
            for index, d in enumerate(api_response):
                for prop in list(d):
                    if type(d[prop]) is dict:
                        for item in list(d[prop]):
                            api_response[index].update({prop + "_" + item: api_response[index][prop][item]})
                        api_response[index].pop(prop)
            results.extend(api_response)
            print("Freshdesk Survey Export has gathered " + str(len(results)) + " Surveys")
        target_results = []
        for index, d in enumerate(results):
            d = {key: value for key, value in results[index].items() if key in properties}
            target_results.append(d)
        df = pd.DataFrame(target_results)
        df.to_csv("raw_freshdesk_surveys.csv", encoding="latin-1", index=False, errors='ignore')
    
    def handle_raw_freshdesk(self, csv_file, general_values, properties_dict, values_dict, date_columns, dtype):
        # Csv into Dataframe
        df = pd.read_csv(csv_file + ".csv", encoding="latin-1")
        # Rename Columns
        df.rename(columns=properties_dict, inplace=True)
        # Date Standarization
        date_columns = [char.lower() for char in date_columns]
        date_columns = ["_".join(char.split(" ")) for char in date_columns]
        for date_column in date_columns:
            df[date_column] = pd.to_datetime(df[df[date_column].notna()][date_column], errors='raise')
            df[date_column] = df[date_column].dt.strftime('%Y-%m-%d %H:%M:%S')
        # print(df.dtypes)
        # Adjust Datatypes
        # for col in list(df.columns):
        #     if col not in date_columns:
        #         df[col] = df[col].astype(dtype[col], errors='ignore')
        # print(df.dtypes)
        # Rename Values
        values_dict.update(general_values)
        df = df.map(lambda x: str(x) if pd.notnull(x) else '')
        df = df.map(lambda x: str(x) if pd.notnull(x) else '')
        df.replace(values_dict, inplace=True)
        # Export
        df.to_csv(csv_file[4:] + ".csv", encoding="latin-1", index=False, errors='ignore')

        # inserta la data resultado de la extraccion en una tabla de la DB postgre
        df.to_sql(csv_file[4:], con=engine, if_exists='replace', index=False)

def dict_into_simple_dict(complex_dict, simple_value):
    headers = list(complex_dict.keys())
    simple_values_list = []
    for k in complex_dict:
        simple_values_list.append(complex_dict[k][simple_value])
    return dict(zip(headers, simple_values_list))


def list_of_datetimes(complex_dict):
    lst = []
    simple_dict = dict_into_simple_dict(complex_dict, "datatype")
    for item in simple_dict:
        if simple_dict[item] == "datetime64":
            lst.append(item)
    return lst

def update_database():
    # DB Schema Handling
    api_schema = json.load(open("api_schema.json"))["API"]
    freshdesk_schema = api_schema["Freshdesk"]
   
    # Freshdesk Schema
    f_tickets_properties = dict_into_simple_dict(freshdesk_schema["Tickets"]["Properties"], "header_name")
    f_tickets_values = freshdesk_schema["Tickets"]["Values"]
    f_tickets_datecolumns = list_of_datetimes(freshdesk_schema["Tickets"]["Properties"])
    f_tickets_datecolumns = [f_tickets_properties[i] for i in f_tickets_datecolumns]
    f_tickets_datatypes = dict(zip(f_tickets_properties.values(),
                                   dict_into_simple_dict(freshdesk_schema["Tickets"]["Properties"],
                                                         "datatype").values()))
    f_surveys_properties = dict_into_simple_dict(freshdesk_schema["Surveys"]["Properties"], "header_name")
    f_surveys_values = freshdesk_schema["Surveys"]["Values"]
    f_surveys_datecolumns = list_of_datetimes(freshdesk_schema["Surveys"]["Properties"])
    f_surveys_datecolumns = [f_surveys_properties[i] for i in f_surveys_datecolumns]
    f_surveys_datatypes = dict(zip(f_surveys_properties.values(),
                                   dict_into_simple_dict(freshdesk_schema["Surveys"]["Properties"],
                                                         "datatype").values()))
    f_generalvalues = freshdesk_schema["General Values"]

    # Class Variables Declaration
    f = FreshdeskAPI()

    # Freshdesk API
    f.raw_export_tickets(f_tickets_properties)
    f.raw_export_surveys(f_surveys_properties)

    f.handle_raw_freshdesk("raw_freshdesk_tickets", f_generalvalues, f_tickets_properties, f_tickets_values,
                           f_tickets_datecolumns, f_tickets_datatypes)
    f.handle_raw_freshdesk("raw_freshdesk_surveys", f_generalvalues, f_surveys_properties, f_surveys_values,
                           f_surveys_datecolumns, f_surveys_datatypes)

update_database()

os.remove("raw_freshdesk_tickets" + ".csv")
os.remove("raw_freshdesk_surveys" + ".csv")
os.remove("freshdesk_tickets" + ".csv")
os.remove("freshdesk_surveys" + ".csv")

Freshdesk Tickets Export has gathered 100 Tickets
Freshdesk Tickets Export has gathered 200 Tickets
Freshdesk Tickets Export has gathered 300 Tickets
Freshdesk Tickets Export has gathered 400 Tickets
Freshdesk Tickets Export has gathered 500 Tickets
Freshdesk Tickets Export has gathered 600 Tickets
Freshdesk Tickets Export has gathered 700 Tickets
Freshdesk Tickets Export has gathered 800 Tickets
Freshdesk Tickets Export has gathered 900 Tickets
Freshdesk Tickets Export has gathered 1000 Tickets
Freshdesk Tickets Export has gathered 1100 Tickets
Freshdesk Tickets Export has gathered 1200 Tickets
Freshdesk Tickets Export has gathered 1300 Tickets
Freshdesk Tickets Export has gathered 1400 Tickets
Freshdesk Tickets Export has gathered 1500 Tickets
Freshdesk Tickets Export has gathered 1600 Tickets
Freshdesk Tickets Export has gathered 1700 Tickets
Freshdesk Tickets Export has gathered 1800 Tickets
Freshdesk Tickets Export has gathered 1900 Tickets
Freshdesk Tickets Export has gathered 20